In [ ]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

## Load data
file = "train_data.csv"
# df_train = pd.read_csv(file, nrows = 40730865/30) # (~ 6 days)
# Total number or rows in train_data: 40730865 (~ 6 months)
df_train = pd.read_csv(file)
df_train = DataFrame(df_train,columns=['custid', 'groupid', 'productid','eventid',\
                                       'currency', 'platform','timestamp', 'invest'])
days = 182

In [ ]:
#( max(df_train['timestamp']) - min(df_train['timestamp']) )/86400
df_train['timestamp'] = df_train['timestamp'] - min(df_train['timestamp'])
df_train = df_train[df_train.timestamp <= (3600*24)*days]
#( max(df_train['timestamp']) - min(df_train['timestamp']) )/86400
df_train['timestamp (day of week)'] = np.digitize(df_train['timestamp'],range(0,3600*24*days,3600*24))
df_train['timestamp (day of week)'] = df_train['timestamp (day of week)'].apply(lambda x: x%7)
df_train['timestamp (hour of day)'] = np.digitize(df_train['timestamp'],range(0,3600*24*days,3600))
df_train['timestamp (hour of day)'] = df_train['timestamp (hour of day)'].apply(lambda x: x%24)

In [ ]:
group_custid_time = df_train['invest'].groupby([df_train['custid'],df_train['timestamp (hour of day)']])
df_custid_time_mean = DataFrame(group_custid_time.mean())
invest_temp = df_custid_time_mean['invest'] 

invest_mean = []
for i in xrange(len(invest_temp)):
    if invest_temp[i] > 15:
        invest_mean.append(15)
    else:    
        invest_mean.append(invest_temp[i])
        
df_custid_time_mean.reset_index(inplace=True) 

In [ ]:
file = "test_data.csv"
df_test = pd.read_csv(file) 

In [ ]:
#( max(df_train['timestamp']) - min(df_train['timestamp']) )/86400
df_test['timestamp'] = df_test['timestamp'] - min(df_test['timestamp'])
df_test = df_test[df_test.timestamp <= (3600*24)*days]
#( max(df_train['timestamp']) - min(df_train['timestamp']) )/86400
df_test['timestamp (day of week)'] = np.digitize(df_test['timestamp'],range(0,3600*24*days,3600*24))
df_test['timestamp (day of week)'] = df_test['timestamp (day of week)'].apply(lambda x: x%7)
df_test['timestamp (hour of day)'] = np.digitize(df_test['timestamp'],range(0,3600*24*days,3600))
df_test['timestamp (hour of day)'] = df_test['timestamp (hour of day)'].apply(lambda x: x%24)

In [ ]:
df_final = pd.merge(df_test,df_custid_time_mean,on=['custid','timestamp (hour of day)'],how='left')
df_final = df_final.fillna(0.0)

#df_final = df_final.fillna(df_final['invest'].mean())

# group_final = df_final['invest'].groupby(df_final['custid'])
# df_= DataFrame(group_final.mean())
# invest_temp = df_['invest'] 

# invest_mean = []
# for i in range(len(invest_temp)):
#     if invest_temp[i] > 15:
#         invest_mean.append(15)
#     else:    
#         invest_mean.append(invest_temp[i])

# custid = DataFrame(df_.index)
# df_2 = DataFrame({'invest_mean':invest_mean})
# df_2['custid'] = custid
# df_2 = df_2[['custid','invest_mean']]
# df_final = pd.merge(df_test,df_2)
# df_final['invest'] = df_final['invest'] + df_final['invest_mean']

In [ ]:
submission_2 = df_final['invest']
submission_2.to_csv('submission_2.csv', index = False)